In [9]:
!pip install -q kaggle torch torchvision tqdm matplotlib


In [10]:
# --- 1️⃣ Download ImageNet-Mini from Kaggle ---
!mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ifigotin/imagenetmini-1000 -p /content
!unzip -q /content/imagenetmini-1000.zip -d /content/imagenet-mini


Dataset URL: https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000
License(s): unknown
100% 3.91G/3.92G [00:53<00:00, 91.3MB/s]
100% 3.92G/3.92G [00:53<00:00, 78.9MB/s]


In [11]:
# --- 2️⃣ Imports & configuration ---
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 1000
BATCH_SIZE = 128
EPOCHS =  100          # can scale to 100 on bigger HW
MAX_LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
LABEL_SMOOTH = 0.1

In [12]:
# --- 3️⃣ Transforms ---
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4,0.4,0.4,0.1),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

In [14]:
# --- 4️⃣ Datasets & Loaders ---
train_ds = datasets.ImageFolder("/content/imagenet-mini/imagenet-mini/train", transform=train_tfms)
val_ds   = datasets.ImageFolder("/content/imagenet-mini/imagenet-mini/val",   transform=val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print(f"✅ Loaded {len(train_ds)} train & {len(val_ds)} val samples.")


✅ Loaded 34745 train & 3923 val samples.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [15]:
# --- 5️⃣ Model, optimizer, scheduler ---
model = resnet50(weights=None, num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
optimizer = optim.SGD(model.parameters(), lr=MAX_LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR,
    steps_per_epoch=len(train_loader), epochs=EPOCHS,
    pct_start=0.3, anneal_strategy='cos',
    div_factor=25.0, final_div_factor=1e4
)

scaler = torch.cuda.amp.GradScaler()


/tmp/ipython-input-608773024.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [16]:
# --- 6️⃣ Training / evaluation ---
def train_one_epoch(model, loader):
    model.train()
    run_loss = 0
    for x,y in tqdm(loader, leave=False):
        x,y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            out = model(x)
            loss = criterion(out,y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        run_loss += loss.item()
    return run_loss/len(loader)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    tot, top1, top5 = 0,0,0
    for x,y in loader:
        x,y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        _,p1 = out.topk(1,1,True,True)
        _,p5 = out.topk(5,1,True,True)
        top1 += (p1.view(-1)==y).sum().item()
        top5 += (p5==y.view(-1,1)).any(dim=1).sum().item()
        tot  += y.size(0)
    return top1/tot, top5/tot

In [ ]:
# --- 7️⃣ Main loop ---
best=0; tr_losses=[]; val_top1s=[]; val_top5s=[]
for ep in range(EPOCHS):
    print(f"\nEpoch {ep+1}/{EPOCHS}")
    tl=train_one_epoch(model,train_loader)
    top1,top5=evaluate(model,val_loader)
    tr_losses.append(tl); val_top1s.append(top1); val_top5s.append(top5)
    print(f"Loss {tl:.4f} | Val@1 {top1*100:.2f}% | Val@5 {top5*100:.2f}%")
    if top1>best:
        best=top1; torch.save(model.state_dict(),"resnet50_imagenetmini_best.pth")
        print(f"✅ Saved new best (Top-1 {best*100:.2f}%)")


Epoch 1/100


  0%|          | 0/272 [00:00<?, ?it/s]/tmp/ipython-input-1292186056.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
 28%|██▊       | 77/272 [01:39<05:36,  1.73s/it]

In [ ]:
# --- 8️⃣ Visualization ---
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(tr_losses); plt.title("Train Loss")
plt.subplot(1,2,2); plt.plot([v*100 for v in val_top1s],label="Top-1")
plt.plot([v*100 for v in val_top5s],label="Top-5"); plt.legend(); plt.title("Validation Acc")
plt.show()
print(f"🏁 Best Top-1 {best*100:.2f}%")